# Exploratory Data Analysis 3
This notebook is designed to explore the structure of the DB1B airline data set. We will examine Coupon data from 2011 Q1.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import signac
from tqdm import tqdm

# Show more columns so we can see everything
pd.options.display.max_columns = 50

project = signac.get_project()

In [2]:
dtypes = {
    'ItinID': np.int64,
    'MktID': np.int64,
    'SeqNum': np.int8,
    'Coupons': np.int8,
    'Year': np.int16,
    'OriginAirportID': np.int16,
    'OriginAirportSeqID': np.int32,
    'OriginCityMarketID': np.int32,
    'Quarter': np.int8,
    'Origin': str,
    'OriginCountry': str,
    'OriginStateFips': np.int8,
    'OriginState': str,
    'OriginStateName': str,
    'OriginWac': np.int8,
    'DestAirportID': np.int16,
    'DestAirportSeqID': np.int32,
    'DestCityMarketID': np.int32,
    'Dest': str,
    'DestCountry': str,
    'DestStateFips': np.int8,
    'DestState': str,
    'DestStateName': str,
    'DestWac': np.int8,
    'Break': str,
    'CouponType': str,
    'TkCarrier': str,
    'OpCarrier': str,
    'RPCarrier': str,
    'Passengers': np.float32,
    'FareClass': str,
    'Distance': np.float32,
    'DistanceGroup': np.int8,
    'ItinGeoType': np.int8,
    'CouponGeoType': np.int8}

In [3]:
job = project.find_jobs({"year": 2011, "quarter": 1}).next()
df = pd.read_csv(job.fn('Coupon.csv'), usecols=dtypes.keys(), dtype=dtypes)

In [4]:
df = df.sort_values(['ItinID', 'SeqNum'])

In [5]:
df.head()

,ItinID,MktID,SeqNum,Coupons,Year,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Quarter,Origin,OriginCountry,OriginStateFips,OriginState,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCountry,DestStateFips,DestState,DestStateName,DestWac,Break,CouponType,TkCarrier,OpCarrier,RPCarrier,Passengers,FareClass,Distance,DistanceGroup,ItinGeoType,CouponGeoType
6444498,201111,20111101,1,1,2011,10135,1013501,30135,1,ABE,US,42,PA,Pennsylvania,23,11057,1105702,31057,CLT,US,37,NC,North Carolina,36,X,A,US,16,16,1.0,X,481.0,1,2,2
6444499,201112,20111201,1,1,2011,10135,1013501,30135,1,ABE,US,42,PA,Pennsylvania,23,11057,1105702,31057,CLT,US,37,NC,North Carolina,36,X,A,US,16,16,1.0,X,481.0,1,2,2
6444500,201113,20111301,1,2,2011,10135,1013501,30135,1,ABE,US,42,PA,Pennsylvania,23,11057,1105702,31057,CLT,US,37,NC,North Carolina,36,X,A,UA,16,16,1.0,X,481.0,1,2,2
7436193,201113,20111302,2,2,2011,11057,1105702,31057,1,CLT,US,37,NC,North Carolina,36,10135,1013501,30135,ABE,US,42,PA,Pennsylvania,23,X,A,UA,16,16,1.0,X,481.0,1,2,2
6444501,201114,20111401,1,2,2011,10135,1013501,30135,1,ABE,US,42,PA,Pennsylvania,23,11057,1105702,31057,CLT,US,37,NC,North Carolina,36,X,A,US,16,16,1.0,X,481.0,1,2,2


In [6]:
df['Passengers'].sort_values(ascending=False)

5089650    964.0
688732     964.0
692203     835.0
5092214    835.0
692000     818.0
700814     740.0
701548     690.0
5099548    690.0
3129981    590.0
2434170    590.0
5090119    579.0
689364     579.0
742452     565.0
6986340    565.0
188079     562.0
1063230    562.0
5092235    544.0
692224     544.0
7051255    543.0
795685     543.0
3364527    538.0
2714858    538.0
2421331    533.0
3327947    533.0
3119684    533.0
2671307    533.0
3084625    530.0
2380499    530.0
2769397    513.0
3411339    513.0
           ...  
5589126      1.0
4839600      1.0
4192814      1.0
4839601      1.0
4839602      1.0
4839603      1.0
5589118      1.0
4839604      1.0
5589119      1.0
4839606      1.0
5589121      1.0
4839607      1.0
5589122      1.0
8210419      1.0
3722502      1.0
4270891      1.0
4839611      1.0
4839608      1.0
5589123      1.0
8210420      1.0
3722503      1.0
4839609      1.0
5589124      1.0
8210421      1.0
3722504      1.0
4839610      1.0
5589125      1.0
8210422      1

In [7]:
col_names = ['ItinID', 'SeqNum', 'OriginAirportID', 'Origin', 'DestAirportID', 'Dest']
df_network = df[col_names]

In [8]:
groups = df_network.groupby(['ItinID'])

In [9]:
def break_itineraries(itin):
    """Breaks discontinuous itineraries into sub-itineraries such that the
    previous destination airport matches the following origin airport."""
    previous_dest = None
    last_index = 0
    for i, coupon in enumerate(itin.itertuples()):
        if previous_dest is not None and previous_dest != coupon.OriginAirportID:
            yield itin.iloc[last_index:i]
            last_index = i
        previous_dest = coupon.DestAirportID
    yield itin.iloc[last_index:]

In [10]:
def itinerary_sequence(itin_id, itin, self_memory=True):
    """Return airport ID sequences."""
    airports = []
    for i, coupon in enumerate(itin.itertuples()):
        if self_memory and i == 0:
            airports.append(coupon.OriginAirportID)
        airports.append(coupon.OriginAirportID)
    airports.append(coupon.DestAirportID)
    if self_memory:
        airports.append(coupon.DestAirportID)
    return itin_id, airports

The code in the cell below works, but there are two major problems:
1. I can't run this directly on the `hadoop-fuse` filesystem for some reason.
2. The code will take forever to run.

I'm going to try to port this notebook to PySpark.

In [11]:
"""
with open('/home/bdice/code/advanced-data-mining-project/airline_data/workspace/6ae284602fc56465315a18f804b30bcb/HON_self_memory.txt',
          'w', buffering=1) as honfile:
    for itin_id, group in tqdm(groups):
        for itin in break_itineraries(group):
            itin_id, airports = itinerary_sequence(itin_id, itin)
            honfile.write('{} {}\n'.format(itin_id, ' '.join(map(str, airports))))
"""

"\nwith open('/home/bdice/code/advanced-data-mining-project/airline_data/workspace/6ae284602fc56465315a18f804b30bcb/HON_self_memory.txt',\n          'w', buffering=1) as honfile:\n    for itin_id, group in tqdm(groups):\n        for itin in break_itineraries(group):\n            itin_id, airports = itinerary_sequence(itin_id, itin)\n            honfile.write('{} {}\n'.format(itin_id, ' '.join(map(str, airports))))\n"